#### Scraping the data from ZINC20 database https://zinc20.docking.org/

All data in this project are coming from a free ZINC20 database. The goal is be to predict logP value based on various, mostly topological, descriptors. 

In this study some of the descriptors were taken directly from ZINC20 molecule pages (example: https://zinc15.docking.org/substances/ZINC000000057058/). These included molecular weight, number of rings, heavy and hetero atoms and the fraction of sp3 hybridized carbon atoms.  Furthermore, for some molecules, additional parameters measured at different pH levels (6.4, 7.0, 7.4, 8.4) were available. Those included: net charge, number of H-bond donors, H-bond acceptors and rotatable bonds, tPSA and apolar and polar desolvation.

Simultaneously, based on SMILES *(Simplified Molecular Input Line Entry Specification)*, which represent the molecules’ structure, with the use of RDKit library additional parameters were obtained. Majority of them were parameters available also from the ZINC database, generated just for the purpose of verification.

For the purpose of web scraping BeautifulSoup library was employed. Scraped data were stored into SQL database.

In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup

Let's create the database with columns names corresponding to the features we are going to scrape.

In [2]:
db = sqlite3.connect('logP.db')
cursor = db.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS molecules (zinc_name text, name text, mwt real, logp real,\
               formula text, rings int, heavy_atoms int, hetero_atoms int, fraction_sp3 real,\
               smiles text, label1 text, charge1 int, Hbond_donors1 int, Hbond_acceptors1 int,\
               tPSA1 real, rotatable_bonds1 int, apolar_des1 real, polar_des1 real, label2 text,\
               charge2 int, Hbond_donors2 int, Hbond_acceptors2 int, tPSA2 real, rotatable_bonds2 int,\
               apolar_des2 real, polar_des2 real, label3 text, charge3 int, Hbond_donors3 int,\
               Hbond_acceptors3 int, tPSA3 real, rotatable_bonds3 int, apolar_des3 real,\
               polar_des3 real, label4 text, charge4 int, Hbond_donors4 int, Hbond_acceptors4 int,\
               tPSA4 real, rotatable_bonds4 int, apolar_des4 real, polar_des4 real, RDmwt real,\
               RDhba int, RDhbd int, RDrotatable int, RDtpsa real, RDcsp3 real, RDCfraction real)")               

Before we scrape the parameters which can be found directly in each molecule's page let's write a function which will allow for getting additional parameters based on molecule's SMILES. This will be done with the use of RDKit library.

In order to use RDKit new environment needs to be prepared: https://www.rdkit.org/docs/Install.html

In [3]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors as descriptors
from rdkit.Chem import rdqueries

def get_RDKit_properties(smiles):
    """
    the function gets selected descriptors based on molecule's SMILES
    if the SMILES is incorrect, all values are returned as NULLs (SQL database friendly format)
    """
    try:
        molecule = Chem.MolFromSmiles(smiles)
    except:
        molecule = None
        RDmwt = None
        RDhba = None
        RDhbd = None
        RDrotatable = None
        RDtpsa = None
        RDcsp3 = None
        RDCfraction = None
    
    if molecule != None:
        # get molecular weight
        try: RDmwt = float(descriptors.CalcExactMolWt(molecule))
        except: RDmwt = None
        # get the number of hydrogen atoms able to engage in hydrogen bonds
        try: RDhbd = int(descriptors.CalcNumHBD(molecule))
        except: RDhbd = None
        # get the number of hetero-atoms able to engage in hydrogen bonds
        try: RDhba = int(descriptors.CalcNumHBA(molecule))
        except: RDhba = None
        # get Topologic Polar Surface Area (the sum of van-Der-Waals surface of polar atoms)
        try: RDtpsa = float(descriptors.CalcTPSA(molecule))
        except: RDtpsa = None
        # get the number of non-terminal sigma-bonds that are not part of a cycle
        try: RDrotatable = int(descriptors.CalcNumRotatableBonds(molecule))
        except: RDrotatable = None
        # get number of sp3 hybridized carbons divided by the total number of carbons in the molecule 
        try: RDcsp3 = float(descriptors.CalcFractionCSP3(molecule))
        except: RDcsp3 = None
        # get fraction of heavy atoms that are carbons 
        try: RDCfraction = float(len(molecule.GetAtomsMatchingQuery(rdqueries.AtomNumEqualsQueryAtom(6)))/molecule.GetNumAtoms())
        except: RDCfraction = None
        
    return RDmwt, RDhba, RDhbd, RDrotatable, RDtpsa, RDcsp3, RDCfraction

Now, that we have a function which gets additional parameters based on molecule's SMILES we can write a function which, for a given molecule page, will:
1. get descriptors which can be directly scraped from the website
2. get SMILES of the molecule and call a function which generates more descriptors based on SMILES
3. write all the descriptors in the database


In [4]:
def get_descriptors(link):
    """
    scrapes all useful information from given ZINC20 molecule webpage: names, descriptors
    in case the values don't exists it replaces them with NULLs (SQL database friendly)
    """
    # get parsed content of given webpage
    response = requests.get(link)
    page_root_element = BeautifulSoup(response.text, 'html.parser')
    
    # get names of molecule:
    names = page_root_element.find('h1').text.split()
    zinc_name = names[0] #1. ZINC20 database name
    if len(names)>1: #2. commercial name
        name = names[1] 
    else: 
        name = None 
    
    # get molecular weight and partition coefficient (logP)
    try: 
        parameters1 = page_root_element.select('table > tbody > tr > td')
    except:
        print('no molecular weight or logP for: ' + link)
    # molecular weight
    try: mwt = float(parameters1[3].text)
    except: mwt = None
    # partition coefficient
    try: logP = float(parameters1[4].text)
    except: logP = None
    
    # get more structural parameters (if available):
    try:
        parameters2 = page_root_element.select('table > tbody > tr')[1].select('td')
    except:
        print('further structural parameters (rings, heavy atoms, etc.) not available for: '+link)
    # get chemical formula    
    try: formula = parameters2[0].text
    except:formula = None
    # get number of rings
    try: rings = parameters2[1].text
    except: rings = None
    # get number of non-hydrogen atoms
    try: heavy_atoms = parameters2[2].text
    except: heavy_atoms = None
    # get number of atoms excluding H anc C
    try: hetero_atoms = parameters2[3].text
    except: hetero_atoms = None
    # get number of sp3 hybridized carbons divided by the total number of carbons in the molecule 
    try: fraction_sp3 = parameters2[4].text
    except: fraction_sp3 = None
      
    # get SMILES
    try:
        smiles = page_root_element.select('.input-sm > input')[0].get('value')
    except:
        smiles = None
        print('SMILES not available for: '+link)
    
    # let's use the SMILES to get additional parameters with RDKit
    RDmwt, RDhba, RDhbd, RDrotatable, RDtpsa, RDcsp3, RDCfraction = get_RDKit_properties(smiles)
    
    # get pH dependent parameters
    try:
        parameters3 = page_root_element.select('.protomers > table > tbody > tr')
   
        # initialize pH dependent parameters with NULLs 
        # (because entries exist not for all pH values: Reference, Low, Mid, High)
        label1 = charge1 = Hbond_donors1 = Hbond_acceptors1 = tPSA1 = rotatable_bonds1 = apolar_des1 = polar_des1 = None
        label2 = charge2 = Hbond_donors2 = Hbond_acceptors2 = tPSA2 = rotatable_bonds2 = apolar_des2 = polar_des2 = None
        label3 = charge3 = Hbond_donors3 = Hbond_acceptors3 = tPSA3 = rotatable_bonds3 = apolar_des3 = polar_des3 = None
        label4 = charge4 = Hbond_donors4 = Hbond_acceptors4 = tPSA4 = rotatable_bonds4 = apolar_des4 = polar_des4 = None
    
        for protomer in parameters3:
            param = protomer.select('td')
            # get pH information
            try: label = param[0].text.strip()
            except: label = None
            # get the overall charge of the molecule at the given pH
            try: charge = int(param[1].text)
            except: charge = None
            # get the number of hydrogen atoms able to engage in hydrogen bonds
            try: Hbond_donors = int(param[2].text)
            except: Hbond_donors = None
            # get the number of hetero-atoms able to engage in hydrogen bonds
            try: Hbond_acceptors = int(param[3].text)
            except: Hbond_acceptors = None
            # get Topologic Polar Surface Area (the sum of van-Der-Waals surface of polar atoms)
            try: tPSA = float(param[4].text)
            except: tPSA = None
            # get the number of non-terminal sigma-bonds that are not part of a cycle
            try: rotatable_bonds = int(param[5].text)
            except: rotatable_bonds = None
            # get apolar desolvation
            try: apolar_des = float(param[6].text)
            except: apolar_des = None
            # get polar desolvation
            try: polar_des = float(param[7].text)
            except: polar_des = None
            
            # assign values to the correct pH         
            if param[0].text.strip().split()[0] == 'Reference':
                label1, charge1, Hbond_donors1, Hbond_acceptors1, tPSA1, rotatable_bonds1, apolar_des1, polar_des1 = label, charge, Hbond_donors, Hbond_acceptors, tPSA, rotatable_bonds, apolar_des, polar_des
            elif param[0].text.strip().split()[0] == 'Low':
                label2, charge2, Hbond_donors2, Hbond_acceptors2, tPSA2, rotatable_bonds2, apolar_des2, polar_des2 = label, charge, Hbond_donors, Hbond_acceptors, tPSA, rotatable_bonds, apolar_des, polar_des
            elif param[0].text.strip().split()[0] == 'Mid':
                label3, charge3, Hbond_donors3, Hbond_acceptors3, tPSA3, rotatable_bonds3, apolar_des3, polar_des3 = label, charge, Hbond_donors, Hbond_acceptors, tPSA, rotatable_bonds, apolar_des, polar_des
            elif param[0].text.strip().split()[0] == 'High':
                label4, charge4, Hbond_donors4, Hbond_acceptors4, tPSA4, rotatable_bonds4, apolar_des4, polar_des4 = label, charge, Hbond_donors, Hbond_acceptors, tPSA, rotatable_bonds, apolar_des, polar_des
        
    except:
        parameters3 = None
        print('no pH dependent parameters for: ' + link)
 
    cursor.execute(
            "INSERT INTO molecules (zinc_name, name, mwt, logp, formula, rings, heavy_atoms, hetero_atoms, fraction_sp3, \
            smiles, label1, charge1, Hbond_donors1, Hbond_acceptors1, tPSA1, rotatable_bonds1, apolar_des1, polar_des1, \
            label2, charge2, Hbond_donors2, Hbond_acceptors2, tPSA2, rotatable_bonds2, apolar_des2, polar_des2, label3, \
            charge3, Hbond_donors3, Hbond_acceptors3, tPSA3, rotatable_bonds3, apolar_des3, polar_des3, label4, charge4, \
            Hbond_donors4, Hbond_acceptors4, tPSA4, rotatable_bonds4, apolar_des4, polar_des4, RDmwt, RDhba, RDhbd, \
            RDrotatable, RDtpsa, RDcsp3, RDCfraction) VALUES (? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,\
            ? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,? ,?, ?, ?, ?, ?, ?, ?, ?)", \
            (zinc_name, name, mwt, logP, formula, rings, heavy_atoms, hetero_atoms, fraction_sp3, smiles, label1, charge1, \
             Hbond_donors1, Hbond_acceptors1, tPSA1, rotatable_bonds1, apolar_des1, polar_des1, label2, charge2, Hbond_donors2, \
             Hbond_acceptors2, tPSA2, rotatable_bonds2, apolar_des2, polar_des2, label3, charge3, Hbond_donors3, Hbond_acceptors3, \
             tPSA3, rotatable_bonds3, apolar_des3, polar_des3, label4, charge4, Hbond_donors4, Hbond_acceptors4, tPSA4, \
             rotatable_bonds4, apolar_des4, polar_des4, RDmwt, RDhba, RDhbd, RDrotatable, RDtpsa, RDcsp3, RDCfraction ))
            
    db.commit()

Now, let's iterate over pages and molecules of ZINC20 website

In [5]:
import time

def get_molecules(number):
    """
    gets molecules and their features from ZINC20 website.
    number - parameter which defines how many molecules funcion will try to scrape.
    """
    # parameters useful if you want to restart data scraping from a specific place
    counter = 0 # before you restart put how many molecules there are currently in the database 
    page_counter = 1 # page of ZINC20 database you want to start from
    
    # continues until defined number of records is scraped into the database
    while counter < number:
        response = requests.get("https://zinc.docking.org/substances/?page="+ str(page_counter), timeout=60)
        page_root_element = BeautifulSoup(response.text, 'html.parser')

        # find links to all molecules presented on this page
        for molecule in page_root_element.find_all('h4'):
            link = molecule.find('a').get("href")
            get_descriptors("https://zinc.docking.org" + link)
            #count already scraped molecules
            counter += 1
        # keep track on which page we are in case connection breaks and there is a need to restart sraping
        page_counter += 1
        print(counter, page_counter)
        
        # delay in order to avoid sending requests to ZINC20 page too often
        time.sleep(10)

I believe that 100 000 molecules will be enough for my project. So let's get the data.

In [6]:
get_molecules(1000)
db.close

100 2
200 3
300 4
400 5
500 6
600 7
700 8
800 9
900 10
1000 11


<function Connection.close()>